# create cluster


eksctl create cluster -f cluster.yaml

https://github.com/pluralsight-cloud/hands-on-with-amazon-eks/blob/main/Infrastructure/eksctl/01-initial-cluster/cluster.yaml


apiVersion: eksctl.io/v1alpha5
kind: ClusterConfig

metadata:
  name: eks-acg
  region: us-east-1
  version: "1.25"

vpc:
  cidr: 10.0.0.0/16
  nat:
    gateway: HighlyAvailable
nodeGroups:
  - name: eks-node-group
    instanceType: t3.medium
    desiredCapacity: 3
    privateNetworking: true
    volumeSize: 30

availabilityZones: ["us-east-1a", "us-east-1b", "us-east-1c"] # This is required since in us-east-1 only 2 subnets are created by default (https://eksctl.io/usage/vpc-networking/)

In [ ]:

kubectl get nodes 

# check out kubernetest 



# create load balancer 

./create 

https://github.com/pluralsight-cloud/hands-on-with-amazon-eks/blob/main/Infrastructure/k8s-tooling/load-balancer-controller/create.sh

after this point you will be able to see your lb controlers in the namespace kube-system but they wont be connected 

kubectl get pods -n kube-system


helm repo add eks https://aws.github.io/eks-charts

helm upgrade --install \
  -n kube-system \
  --set clusterName=eks-acg \
  --set serviceAccount.create=true \
  aws-load-balancer-controller eks/aws-load-balancer-controller

aws cloudformation deploy \
    --stack-name aws-load-balancer-iam-policy \
    --template-file iam-policy.yaml \
    --capabilities CAPABILITY_IAM

# iam policy to nodeInstanceRole 

# deployment 

apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
  namespace: {{ .Release.Namespace }}
  labels:
    app: nginx
spec:
  replicas: 3
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx:1.14.2
        ports:
        - containerPort: 80

# ingress 

apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: services-ingress
  namespace: {{ .Release.Namespace }}
  annotations:
    kubernetes.io/ingress.class: alb
    alb.ingress.kubernetes.io/group.name: my-group
    alb.ingress.kubernetes.io/scheme: internet-facing
{{- if not .Values.ssl.enabled }}
    alb.ingress.kubernetes.io/listen-ports: '[{"HTTP": 80}]'
{{- else }}
    alb.ingress.kubernetes.io/ssl-redirect: '443'
    alb.ingress.kubernetes.io/listen-ports: '[{"HTTP": 80}, {"HTTPS": 443}]'
{{- end }}
  labels:
    app: nginx
spec:
  rules:
    - http:
        paths:
          - path: /*
            pathType: ImplementationSpecific
            backend:
              service:
                name: nginx-service
                port: 
                  number: 80
{{- if .Values.ssl.enabled }}
      host: sample-app.{{ .Values.baseDomain }}
{{- end }}


# service 

apiVersion: v1
kind: Service
metadata:
  name: nginx-service
  namespace: {{ .Release.Namespace }}
spec:
  type: NodePort
  selector:
    app: nginx
  ports:
    - protocol: TCP
      port: 80
      targetPort: 80




# use above with https://github.com/pluralsight-cloud/hands-on-with-amazon-eks/blob/main/Infrastructure/k8s-tooling/load-balancer-controller/test/run.sh

helm upgrade --install sample-app .